Phase 1: Install required libraries

In [28]:
!pip install openai flask flask-cors gradio tiktoken numpy
# If using FAISS for retrieval, also install: faiss-cpu
# pip install faiss-cpu

In [29]:
!pip install faiss-cpu

In [30]:
!pip install requests PyPDF2 python-dotenv

In [39]:
!pip install sentence-transformers

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.0 MB 13.1 MB/s eta 0:00:01
   -------------- ------------------------- 4.5/12.0 MB 12.2 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.0 MB 12.3 MB/s eta 0:00:01
   ------------------------------ --------- 9.2/12.0 MB 11.4 MB/s eta 0:00:01
   -------------------------------------- - 11.5/12.0 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 11.2 MB/s  0:00:01
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 3.9 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------------------------------- -------- 2.1/2.7 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 9.7 MB/s  0:00:00

  Attempting uninstall: huggingface-hub

    Found existing installation: huggingface_hub 1.1.4

Phase 2: Initialize Client

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="API KEY")
print("Client initialized")

Client initialized


Phase 3: Load HuggingFace Embedding Model

PHASE 4 — Utility functions

In [33]:
import re

def redact_pii(text):
    text = re.sub(r"\b\d{10}\b", "[REDACTED_PHONE]", text)
    text = re.sub(r"\b\d{12}\b", "[REDACTED_ID]", text)
    text = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", "[REDACTED_EMAIL]", text)
    return text

def chunk_text(text, max_words=250):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

# Splits long documents into 250-word chunks
# Removes personal information before sending to AI

LLM Helper Functions: Moderation + Embeddings + Generate

In [35]:
def moderate(text):
    resp = client.moderations.create(
        model="omni-moderation-latest",
        input=text
    )
    return resp.results[0].flagged

def embed_text(texts):
    resp = client.embeddings.create(
        model="text-embedding-3-large",
        input=texts
    )
    return [d.embedding for d in resp.data]

def embed_query(q):
    resp = client.embeddings.create(
        model="text-embedding-3-large",
        input=[q]
    )
    return resp.data[0].embedding

def generate(messages):
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0
    )
    return resp.choices[0].message.content


#Embeds documents
#Embeds user queries
#Safely generates responses
#Moderates inappropriate questions

Create sample data_txt/ folder and sample file

In [36]:
from pathlib import Path

Path("data_txt").mkdir(exist_ok=True)

sample_text = """Tamil Nadu Government FAQ

Widow Pension:
Apply at Taluk Office. Need death certificate + ID proof.

Driving Licence Renewal:
Renew through Parivahan. Need existing DL + address proof.

Community Certificate:
Apply at e-Sevai or Tahsildar office with address proof and TC.
"""

Path("data_txt/tn_faq.txt").write_text(sample_text, encoding="utf-8")

print("Sample document created!")


Sample document created!


Build FAISS Vector Index

This is DOCUMENT INGESTION.

Define Retrieval Function

In [ ]:
def retrieve(query, k=5):
    index = faiss.read_index("faiss_index.idx")
    meta = json.load(open("faiss_meta.json", "r", encoding="utf-8"))

    q_vec = np.array(embed_query(query), dtype="float32").reshape(1, -1)

    D, I = index.search(q_vec, k)

    results = []
    for idx in I[0]:
        results.append(meta[idx])
    return results

#Retrieves top 5 most similar chunks to user query

Build Prompt with Context

In [ ]:
def build_prompt(query, contexts):
    ctx_text = "\n---\n".join([c["text"] for c in contexts])

    return [
        {
            "role": "system",
            "content": (
                "You are the Tamil Nadu Government Assistant.\n"
                "Answer ONLY using the context.\n"
                "If answer is not in context, respond:\n"
                "\"Information not available. Visit https://www.tn.gov.in/\""
            )
        },
        {
            "role": "user",
            "content": f"Question: {query}\n\nContext:\n{ctx_text}"
        }
    ]


#Ensures the model answers using ONLY retrieved data
#No hallucination
#Context-aware

Final Pipeline Function

In [ ]:
def answer_question(query):
    cleaned = redact_pii(query)

    if moderate(cleaned):
        return "This request cannot be processed."

    ctx = retrieve(cleaned)
    messages = build_prompt(cleaned, ctx)
    reply = generate(messages)

    return reply


Test the System

In [ ]:
print(answer_question("How to apply for widow pension in Tamil Nadu?"))


Inline Gradio UI

In [37]:
import gradio as gr

def chat_fn(message):
    return answer_question(message)

ui = gr.Interface(
    fn=chat_fn,
    inputs="text",
    outputs="text",
    title="TN Government Info Assistant",
)

ui.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
